# SKN13-3rd-3Team: Data Preprocessing

1. Preprocessing

2. ~~Sentiment Analysis for metadata~~

In [2]:
import pandas as pd

# 각자 파일 경로에 맞게 적용해주세요 :)
df = pd.read_csv('oliveyoung_lotion_prep.csv')
df.head()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 75085 entries, 0 to 75084
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   제품명     75085 non-null  object
 1   성분      75085 non-null  object
 2   별점      75085 non-null  int64 
 3   피부타입    75085 non-null  object
 4   피부고민    75085 non-null  object
 5   자극도     75085 non-null  object
 6   리뷰      75085 non-null  object
 7   카테고리    75085 non-null  object
dtypes: int64(1), object(7)
memory usage: 4.6+ MB


## Preprocessing: cleansing, outliers, and missing values

- 결측치를 제거합니다.
- 제품명에 들어간 불필요한 요소를 전처리합니다.
    - ‘리필’ 또는 ‘기획’이 들어간 경우 데이터 포인트 자체를 제거합니다.
    - 괄호(`[]`, `()` 등)와 괄호 안의 문자열을 제거합니다.
    - 제품명에 증정품 관련 문자열이 있는 경우 제거합니다.
        > e.g.
        >    
        >    ‘A사 a 로션 + 토너 50ml’의 경우 ‘ + 토너 50ml’를 제품명에서 제거합니다.
            
- 리뷰가 100개 미만인 제품은 이상치로 간주하여 제거합니다.
- 불필요한 열을 제거합니다. **이 부분은 같이 얘기해봐요**
- 리뷰를 문장 단위로 split합니다.
    - `kss.split_sentences`
    - 분리된 문장만큼 행을 확장합니다
        > e.g.
        >
        > A 제품의 리뷰들 중 k번째 리뷰는 4개의 문장으로 구성되어 있습니다.
        >
        > 이 리뷰가 차지하는 행은 1개였지만, split이 완료되면 4개가 됩니다.

In [3]:
# 일부 csv 파일들은 결측치가 남아있습니다.
df.isna().sum()

제품명     0
성분      0
별점      0
피부타입    0
피부고민    0
자극도     0
리뷰      0
카테고리    0
dtype: int64

In [4]:
# 결측치 대체 후 잘 대체됐는지 확인하기 위해 인덱스를 저장해둡니다.
na_index = df[df['성분'].isna()].index

In [5]:
df = df.fillna('정보없음')

# 저장해둔 인덱스로 조회해서 확인합니다.
df.iloc[na_index,:]

,제품명,성분,별점,피부타입,피부고민,자극도,리뷰,카테고리


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 74988 entries, 0 to 75084
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   제품명     74988 non-null  object
 1   성분      74988 non-null  object
 2   별점      74988 non-null  int64 
 3   피부타입    74988 non-null  object
 4   피부고민    74988 non-null  object
 5   자극도     74988 non-null  object
 6   리뷰      74988 non-null  object
 7   카테고리    74988 non-null  object
dtypes: int64(1), object(7)
memory usage: 5.1+ MB


In [6]:
# 일부 csv 파일에 리뷰가 100개 미만인 제품이 남아있어 해당 제품을 이상치로 간주하고 제거합니다.
counts = df['제품명'].value_counts()
over100 = counts[counts >= 100].index.tolist()
df = df[df['제품명'].isin(over100)]
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 74988 entries, 0 to 75084
Data columns (total 8 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   제품명     74988 non-null  object
 1   성분      74988 non-null  object
 2   별점      74988 non-null  int64 
 3   피부타입    74988 non-null  object
 4   피부고민    74988 non-null  object
 5   자극도     74988 non-null  object
 6   리뷰      74988 non-null  object
 7   카테고리    74988 non-null  object
dtypes: int64(1), object(7)
memory usage: 5.1+ MB


In [7]:
df['제품명'].value_counts().describe()

count     115.000000
mean      652.069565
std       517.067859
min       104.000000
25%       240.000000
50%       510.000000
75%       981.000000
max      3274.000000
Name: count, dtype: float64

In [9]:
import kss
from tqdm.auto import tqdm

# 문장단위로 
tqdm.pandas(desc='Tokenizing...')
df['문장리스트'] = df['리뷰'].progress_apply(kss.split_sentences)
df = df.explode('문장리스트', ignore_index=True)
df = df.drop(columns=['리뷰'])
df = df.rename(columns={'문장리스트':'리뷰'})
df.head(3)

Tokenizing...:   0%|          | 0/74988 [00:00<?, ?it/s]

[Kss]: Oh! You have mecab in your environment. Kss will take this as a backend! :D



,제품명,성분,별점,피부타입,피부고민,자극도,카테고리,리뷰
0,아이디얼포맨 퍼펙트 올인원,"[퍼펙트올인원]\n정제수,글리세린,에틸헥실스테아레이트,1,2-헥산다이올,메틸프로판다...",5,복합성에 좋아요,보습에 좋아요,자극없이 순해요,로션,제품 구성이 괜찮아서 지인 선물용으로 재구매했어요.
1,아이디얼포맨 퍼펙트 올인원,"[퍼펙트올인원]\n정제수,글리세린,에틸헥실스테아레이트,1,2-헥산다이올,메틸프로판다...",5,복합성에 좋아요,보습에 좋아요,자극없이 순해요,로션,클렌징품이 선크림까지 지워지는 제품이라서 좋아요.
2,아이디얼포맨 퍼펙트 올인원,"[퍼펙트올인원]\n정제수,글리세린,에틸헥실스테아레이트,1,2-헥산다이올,메틸프로판다...",5,복합성에 좋아요,보습에 좋아요,자극없이 순해요,로션,피부 보습이나 전체적인 사용감이 괜찮아서 지인 선물용으로 종종 구매해요.


In [10]:
# 확인용
print(df.shape)
df['리뷰'][:10]

(234981, 8)


0                제품 구성이 괜찮아서 지인 선물용으로 재구매했어요.
1                 클렌징품이 선크림까지 지워지는 제품이라서 좋아요.
2    피부 보습이나 전체적인 사용감이 괜찮아서 지인 선물용으로 종종 구매해요.
3                     올인원이라 사용 간편하고 괜찮은것 같아요!
4                          가볍게 쓰기 좋은 남자화장품이에요
5             남성분들이 쓰기에 무난한 제품이라 재재재구매한 제품입니다
6           할아버지 사드렸는데 무난하게 잘 사용하고 계신 것 같아요 !
7                    연령대 상관없이 잘 쓸 수 있는 것 같습니당
8                              촉촉하이 깔끔하고 좋네예~
9                          근데 면도한데 바른게 따끔거리예~
Name: 리뷰, dtype: object

### 문장 단위로 split된 DataFrame을 'oliveyoung_카테고리명_chunked.csv'로 저장해주세요. 고맙습니다!

In [ ]:
df.to_csv('oliveyoung_lotion_tokenized.csv', index=False, encoding='utf-8')